In [1]:
# Required Modules
import pandas as pd
import numpy as np
import lazypredict
import warnings
warnings.filterwarnings("ignore")

from scipy import stats

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from math import sqrt


from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier


In [7]:
data = pd.read_csv('../../Resources/Clean_Data/Final_Data.csv')
data.head()

,Zip_Code,Date,Date_Code,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,201901,2019,1,244950.00,4.46,3778,1113,94154.00,...,10671,1069,104,35,28,195960.00,360.00,1.00,988.71,0.13
1,32003,2019-02-01,201902,2019,2,270000.00,4.37,3778,1113,94154.00,...,10671,1069,104,35,28,216000.00,360.00,1.00,1077.82,0.14
2,32003,2019-03-01,201903,2019,3,275000.00,4.26,3778,1113,94154.00,...,10671,1069,104,35,28,220000.00,360.00,1.00,1084.20,0.14
3,32003,2019-04-01,201904,2019,4,264500.00,4.14,3778,1113,94154.00,...,10671,1069,104,35,28,211600.00,360.00,1.00,1027.67,0.13
4,32003,2019-05-01,201905,2019,5,281000.00,4.07,3778,1113,94154.00,...,10671,1069,104,35,28,224800.00,360.00,1.00,1082.58,0.14


In [8]:
ml_data = data[['Zip_Code', 'Year', 'Month', 'Mobility_Rate', 'Expense_Index', 'Crime_Index' , 'Total_Vacant', 'Total_Dwellings', 'Total_Sales', 'FHA_Count', 'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
# ml_data.index = ml_data['Zip_Code']
ml_data.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [9]:
ml_data_3mo = ml_data.sort_values(by = ['Zip_Code', 'Year', 'Month'], ascending = [True, True, True])
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00


In [10]:
ml_data_3mo['Last_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift()
ml_data_3mo['Last_2Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(2)
ml_data_3mo['Last_3Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(3)


price_difference_1 = []
price_difference_2 = []
price_difference_3 = []


for column in ml_data_3mo[['Sale_Price', 'Last_Month_Price', 'Last_2Month_Price',
                    'Last_3Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = ml_data_3mo['Sale_Price'].values
    last_sales = ml_data_3mo['Last_Month_Price'].values
    last_2mo_sale = ml_data_3mo['Last_2Month_Price'].values
    last_3mo_sale = ml_data_3mo['Last_3Month_Price'].values
    price_difference_1.append(current_sales-last_sales)
    price_difference_2.append(last_sales-last_2mo_sale)
    price_difference_3.append(last_2mo_sale-last_3mo_sale)
    

ml_data_3mo['Last_Month_Diff'] = price_difference_1[0]
ml_data_3mo['Last_2Month_Diff'] = price_difference_2[0]
ml_data_3mo['Last_3Month_Diff'] = price_difference_3[0]

ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
0,32003,2019,1,0.18,104,28,704,11724,32,7,0.13,0.14,244950.00,nan,nan,nan,nan,nan,nan
1,32003,2019,2,0.18,104,28,704,11724,41,2,0.14,0.14,270000.00,244950.00,nan,nan,25050.00,nan,nan
2,32003,2019,3,0.18,104,28,704,11724,21,7,0.14,0.14,275000.00,270000.00,244950.00,nan,5000.00,25050.00,nan
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00


In [11]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)
ml_data_3mo.head()

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00
5,32003,2019,6,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00,281000.00,264500.00,275000.00,-6000.00,16500.00,-10500.00
6,32003,2019,7,0.18,104,28,704,11724,90,7,0.13,0.14,282500.00,275000.00,281000.00,264500.00,7500.00,-6000.00,16500.00
7,32003,2019,8,0.18,104,28,704,11724,64,5,0.13,0.14,269577.00,282500.00,275000.00,281000.00,-12923.00,7500.00,-6000.00


In [12]:
y_value = ml_data_3mo['Sale_Price'].values

x_values = ml_data_3mo.drop(columns='Sale_Price')

feature_names = x_values.columns

x_values

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00
5,32003,2019,6,0.18,104,28,704,11724,43,3,0.13,0.14,281000.00,264500.00,275000.00,-6000.00,16500.00,-10500.00
6,32003,2019,7,0.18,104,28,704,11724,90,7,0.13,0.14,275000.00,281000.00,264500.00,7500.00,-6000.00,16500.00
7,32003,2019,8,0.18,104,28,704,11724,64,5,0.13,0.14,282500.00,275000.00,281000.00,-12923.00,7500.00,-6000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23505,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,285000.00,256000.00,251000.00,-25250.00,29000.00,5000.00
23506,34997,2020,12,0.14,91,113,2976,21551,142,16,0.20,0.25,259750.00,285000.00,256000.00,37250.00,-25250.00,29000.00
23507,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,297000.00,259750.00,285000.00,8000.00,37250.00,-25250.00
23508,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,305000.00,297000.00,259750.00,-5000.00,8000.00,37250.00


In [13]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [14]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
reg_models, reg_predictions = reg.fit(X_train, X_test, y_train, y_test)

print(reg_models)

100%|██████████| 42/42 [01:58<00:00,  2.83s/it]                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
OrthogonalMatchingPursuitCV                  1.00       1.00      0.00   
HuberRegressor                               1.00       1.00      0.00   
TransformedTargetRegressor                   1.00       1.00      0.00   
LarsCV                                       1.00       1.00      0.00   
Lars                                         1.00       1.00      0.00   
RANSACRegressor                              1.00       1.00      0.00   
LassoLarsCV                                  1.00       1.00      0.00   
LassoLarsIC                                  1.00       1.00      0.00   
LinearRegression                             1.00       1.00      0.00   
PassiveAggressiveRegressor                   1.00       1.00      0.02   
RidgeCV                                      1.00       1.00     

In [17]:
pip install dataframe-image

     |████████████████████████████████| 647 kB 4.4 MB/s 
     |████████████████████████████████| 124 kB 22.0 MB/s 
     |████████████████████████████████| 49 kB 19.5 MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=0afb6426cc64115db02b7e169d40b869309e4da3c1d2449c173c14725f7cf136
  Stored in directory: /Users/jjwilliams/Library/Caches/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built idna-ssl
Note: you may need to restart the kernel to use updated packages.


In [18]:
import dataframe_image as dfi
dfi.export(reg_models, 'lazy_pred_reg.png')

In [20]:
price_change_1mo = []

for row in ml_data_3mo['Last_Month_Diff']:
    if row > 0:
        price_change_1mo.append(1)
    else:
        price_change_1mo.append(0)
ml_data_3mo['Price_Change_1mo'] = price_change_1mo

ml_data_3mo

,Zip_Code,Year,Month,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,Price_Change_1mo
3,32003,2019,4,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,275000.00,270000.00,244950.00,-10500.00,5000.00,25050.00,0
4,32003,2019,5,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,264500.00,275000.00,270000.00,16500.00,-10500.00,5000.00,1
5,32003,2019,6,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00,281000.00,264500.00,275000.00,-6000.00,16500.00,-10500.00,0
6,32003,2019,7,0.18,104,28,704,11724,90,7,0.13,0.14,282500.00,275000.00,281000.00,264500.00,7500.00,-6000.00,16500.00,1
7,32003,2019,8,0.18,104,28,704,11724,64,5,0.13,0.14,269577.00,282500.00,275000.00,281000.00,-12923.00,7500.00,-6000.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23505,34997,2020,11,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00,285000.00,256000.00,251000.00,-25250.00,29000.00,5000.00,0
23506,34997,2020,12,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00,259750.00,285000.00,256000.00,37250.00,-25250.00,29000.00,1
23507,34997,2021,1,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00,297000.00,259750.00,285000.00,8000.00,37250.00,-25250.00,1
23508,34997,2021,2,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00,305000.00,297000.00,259750.00,-5000.00,8000.00,37250.00,0


In [21]:
ml_data_3mo = ml_data_3mo.drop(columns=['Zip_Code', 'Year', 'Month','Last_Month_Price', 'Last_Month_Diff', 'Last_2Month_Price', 'Last_2Month_Diff', 'Last_3Month_Price', 'Last_3Month_Diff'])
ml_data_3mo.head()

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Price_Change_1mo
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00,0
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00,1
5,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00,0
6,0.18,104,28,704,11724,90,7,0.13,0.14,282500.00,1
7,0.18,104,28,704,11724,64,5,0.13,0.14,269577.00,0


In [22]:
ml_data_3mo.replace([np.inf, -np.inf], np.nan, inplace=True)
ml_data_3mo.dropna(inplace=True)

In [23]:
y_value = ml_data_3mo['Price_Change_1mo'].values

x_values = ml_data_3mo.drop(columns='Price_Change_1mo')

feature_names = x_values.columns

x_values

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
3,0.18,104,28,704,11724,42,3,0.13,0.14,264500.00
4,0.18,104,28,704,11724,66,8,0.14,0.14,281000.00
5,0.18,104,28,704,11724,43,3,0.13,0.14,275000.00
6,0.18,104,28,704,11724,90,7,0.13,0.14,282500.00
7,0.18,104,28,704,11724,64,5,0.13,0.14,269577.00
...,...,...,...,...,...,...,...,...,...,...
23505,0.14,91,113,2976,21551,132,11,0.17,0.25,259750.00
23506,0.14,91,113,2976,21551,142,16,0.20,0.25,297000.00
23507,0.14,91,113,2976,21551,81,10,0.20,0.25,305000.00
23508,0.14,91,113,2976,21551,116,7,0.20,0.25,300000.00


In [24]:
X_train, X_test, y_train, y_test = train_test_split(x_values, y_value, random_state=42)

In [25]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=False)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
models

 21%|██        | 6/29 [00:05<00:17,  1.28it/s]CategoricalNB model failed to execute
Negative values in data passed to CategoricalNB (input X)
 90%|████████▉ | 26/29 [01:23<00:11,  3.86s/it]StackingClassifier model failed to execute
__init__() missing 1 required positional argument: 'estimators'
100%|██████████| 29/29 [01:24<00:00,  2.92s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.67,0.67,0.67,0.67,2.55
ExtraTreesClassifier,0.66,0.66,0.66,0.66,1.50
LGBMClassifier,0.66,0.65,0.65,0.66,0.30
XGBClassifier,0.66,0.65,0.65,0.65,0.59
BaggingClassifier,0.64,0.64,0.64,0.64,0.72
NuSVC,0.64,0.64,0.64,0.64,48.47
SVC,0.63,0.62,0.62,0.63,7.34
CalibratedClassifierCV,0.61,0.61,0.61,0.61,3.70
LinearSVC,0.61,0.61,0.61,0.61,0.99


In [26]:
dfi.export(models, 'lazy_pred_clas.png')